In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from malaya_speech.torch_model.vits_v2.model_infer import SynthesizerTrn
from malaya_boilerplate.train.config import HParams
import json

2023-08-26 21:19:14.127608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 21:19:14.215080: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-26 21:19:14.794171: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-26 21:19:14.794203: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
config = 'malay_vits/jordan.json'

In [4]:
with open(config) as fopen:
    hps = HParams(**json.load(fopen))

In [5]:
from malaya_speech.utils.text import TTS_SYMBOLS
import torch

In [6]:
model = SynthesizerTrn(len(TTS_SYMBOLS),
                                   hps.data.n_mel_channels,
                                   hps.train.segment_size // hps.data.hop_length,
                                   **hps.model)

In [36]:
!ls malay_vits/logs/jordan-v2/G_*.pth

malay_vits/logs/jordan-v2/G_0.pth      malay_vits/logs/jordan-v2/G_50000.pth
malay_vits/logs/jordan-v2/G_10000.pth  malay_vits/logs/jordan-v2/G_5000.pth
malay_vits/logs/jordan-v2/G_15000.pth  malay_vits/logs/jordan-v2/G_55000.pth
malay_vits/logs/jordan-v2/G_20000.pth  malay_vits/logs/jordan-v2/G_60000.pth
malay_vits/logs/jordan-v2/G_25000.pth  malay_vits/logs/jordan-v2/G_65000.pth
malay_vits/logs/jordan-v2/G_30000.pth  malay_vits/logs/jordan-v2/G_70000.pth
malay_vits/logs/jordan-v2/G_35000.pth  malay_vits/logs/jordan-v2/G_75000.pth
malay_vits/logs/jordan-v2/G_40000.pth  malay_vits/logs/jordan-v2/G_80000.pth
malay_vits/logs/jordan-v2/G_45000.pth


In [37]:
pth = 'malay_vits/logs/jordan-v2/G_80000.pth'

In [38]:
model.load_state_dict(torch.load(pth, map_location=torch.device('cpu'))['model'])

<All keys matched successfully>

In [39]:
_ = model.eval()

In [40]:
from malaya_speech.torch_model.vits.commons import intersperse
import malaya_speech

In [41]:
normalizer = malaya_speech.utils.text.TextIDS(pad_to = None, is_lower = False)

In [42]:
text = 'a family of large language models for code based on'

In [43]:
t, ids = normalizer.normalize(text)
t

'a family of large language models for code based on .'

In [44]:
ids = intersperse(ids, 0)

In [45]:
ids = torch.LongTensor(ids)
ids_lengths = torch.LongTensor([ids.size(0)])
ids = ids.unsqueeze(0)

In [49]:
audio = model.infer(
    ids,
    ids_lengths,
    noise_scale=0.666,
    noise_scale_w=0.666,
    length_scale=1.0,
)

In [50]:
audio = audio[0][0,0].detach().numpy()

In [51]:
import IPython.display as ipd
ipd.Audio(audio, rate = 22050)

In [52]:
torch.save(model.state_dict(), 'jordan.pth')

In [53]:
from malaya_boilerplate.huggingface import upload_dict

In [ ]:
files_mapping = {'jordan.pth': 'model.pth',
                config: 'config.json'}
upload_dict(model = 'VITS-V2-jordan', files_mapping = files_mapping, username = 'mesolitica')